#### **Name:** Fatima Javaid
#### **Roll no:** 2024-csr-010
#### **AI Maze Game**

## 1. CREATE TXT FILE

In [3]:
##########
#A   #   #
# ## # # #
#    # #B#
##########

## 2. CODE FOR MAZE

In [8]:
# ---------- Professional Maze Solver (reads external maze.txt) ----------

# Yeh dataclasses ka import hai jahan se hum simple data structures banayenge
from dataclasses import dataclass
# Yeh typing se aata hai taake hum type hints use kar saken (List, Tuple, Optional)
from typing import List, Tuple, Optional
# Yeh matplotlib hai jo maze ko graphical form mein show karega
import matplotlib.pyplot as plt
# Yeh numpy hai jo arrays banane aur unpe operations ke liye use hota hai
import numpy as np

# ---------------- Node + Frontiers ----------------
@dataclass   # Yeh decorator simple class ko ek data container bana deta hai
class Node:
    state: Tuple[int, int]     # Maze ke andar ek (row, column) position ko store karega
    parent: Optional['Node']   # Yeh batayega ke yeh node kis parent node se aayi
    action: Optional[str]      # Yeh string hogi jo move ka naam rakhegi (up, down, left, right)


class StackFrontier:
    """DFS Frontier (stack)"""
    def _init_(self):
        self.frontier: List[Node] = []   # Yeh ek list hai jo stack ka kaam karegi

    def add(self, node: Node):
        self.frontier.append(node)       # Yeh ek node ko stack ke andar dalta hai (append karta hai)

    def contains_state(self, state: Tuple[int, int]) -> bool:
        return any(node.state == state for node in self.frontier)   # Yeh check karta hai ke kya given state already frontier mein hai

    def empty(self) -> bool:
        return len(self.frontier) == 0   # Agar list empty ho toh True return karega

    def remove(self) -> Node:
        if self.empty():
            raise Exception('Frontier is empty')   # Agar khali hai toh error
        node = self.frontier[-1]   # Stack ka last element nikalna hai (LIFO)
        self.frontier = self.frontier[:-1]   # Last element ko list se remove kar diya
        return node


class QueueFrontier(StackFrontier):
    """BFS Frontier (queue)"""
    def remove(self) -> Node:
        if self.empty():
            raise Exception('Frontier is empty')   # Agar khali hai toh error
        node = self.frontier[0]   # Queue ka pehla element nikalna hai (FIFO)
        self.frontier = self.frontier[1:]   # First element remove kar diya
        return node


# ---------------- Maze Solver ----------------
class Maze:
    def __init__(self, lines: List[str]):
        contents = lines   # Maze ki lines (jo file se aayengi) hum yahan rakhte hain

        # Yeh check kar raha hai ke sirf ek hi start point ho (A)
        if sum(line.count('A') for line in contents) != 1:
            raise Exception("Maze must have exactly one start point 'A'")
        # Yeh check kar raha hai ke sirf ek hi goal point ho (B)
        if sum(line.count('B') for line in contents) != 1:
            raise Exception("Maze must have exactly one goal point 'B'")

        self.height = len(contents)    # Maze ki kitni rows hain
        self.width = max(len(line) for line in contents)   # Sabse badi row ka length (columns ki max width)
        self.walls = []    # Yeh 2D array banega jo batayega kahan walls hain aur kahan path hai
        self.start = None  # Yeh tuple (row, col) rakhega jahan se start hai
        self.goal = None   # Yeh tuple (row, col) rakhega jahan goal hai

        # Parse maze layout (line by line maze ko samajhna)
        for i in range(self.height):
            row = []   # Ek nayi row list ban gayi
            for j in range(self.width):
                try:
                    ch = contents[i][j]   # Har cell ko dekh rahe hain
                except IndexError:
                    ch = ' '   # Agar row choti ho toh usko blank samajh lo
                if ch == 'A':   # Agar yeh start point hai
                    self.start = (i, j)
                    row.append(False)   # Wall nahi hai
                elif ch == 'B':   # Agar yeh goal hai
                    self.goal = (i, j)
                    row.append(False)
                elif ch == ' ':   # Agar empty space hai toh wall nahi hai
                    row.append(False)
                else:
                    row.append(True)   # Baaki sab ko wall samajh lo
            self.walls.append(row)   # Is row ko walls list mein add kar do

        self.solution = None    # Yeh solution path rakhega
        self.explored = set()   # Yeh set rakhega jitne cells explore kiye gaye
        self.num_explored = 0   # Yeh counter rakhega total explored cells ka number

    @classmethod
    def from_file(cls, filename: str) -> 'Maze':
        with open(filename) as f:
            lines = [line.rstrip('\n') for line in f if line.strip()]   # Maze file se sari lines le aate hain
        return cls(lines)

    def neighbors(self, state):
        row, col = state   # Current position ko tod kar row aur column nikal liya
        # Yeh 4 possible moves hain (up, down, left, right)
        candidates = [
            ('up', (row - 1, col)),
            ('down', (row + 1, col)),
            ('left', (row, col - 1)),
            ('right', (row, col + 1)),
        ]
        result = []
        for action, (r, c) in candidates:
            # Agar yeh move maze ke andar hai aur wall nahi hai toh valid hai
            if 0 <= r < self.height and 0 <= c < self.width and not self.walls[r][c]:
                result.append((action, (r, c)))
        return result

    def solve(self, frontier='bfs'):
        if frontier not in ('dfs', 'bfs'):
            raise ValueError("Frontier must be 'dfs' or 'bfs'")

        start_node = Node(state=self.start, parent=None, action=None)   # Starting node banayi
        container = StackFrontier() if frontier == 'dfs' else QueueFrontier()   # DFS ya BFS choose kar lo
        container.add(start_node)

        self.explored = set()   # Reset explored
        self.num_explored = 0   # Reset counter

        while True:
            if container.empty():
                raise Exception('No solution found')   # Agar container empty ho gaya toh solution nahi mila

            node = container.remove()   # Ek node nikali
            self.num_explored += 1   # Counter badhaya

            if node.state == self.goal:   # Agar goal mil gaya toh solution trace karna shuru karo
                actions, cells = [], []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)   # Solution save kar liya
                return

            self.explored.add(node.state)   # Is node ko explored set mein add kar diya

            for action, state in self.neighbors(node.state):   # Har neighbor check karo
                if not container.contains_state(state) and state not in self.explored:
                    container.add(Node(state=state, parent=node, action=action))   # Naye nodes frontier mein add karo

    def _render_array(self, show_solution=True, show_explored=False, cell_size=20):
        h, w = self.height, self.width
        canvas = np.zeros((h, w, 3), dtype=np.uint8) + 237   # Yeh ek numpy array ban gaya jisme background light grey hai

        # Colors define kar rahe hain
        wall_color = np.array([40, 40, 40], dtype=np.uint8)
        start_color = np.array([255, 0, 0], dtype=np.uint8)
        goal_color = np.array([0, 171, 28], dtype=np.uint8)
        solution_color = np.array([220, 235, 113], dtype=np.uint8)
        explored_color = np.array([212, 97, 85], dtype=np.uint8)

        solution_cells = set(self.solution[1]) if (self.solution is not None and show_solution) else set()

        for i in range(h):
            for j in range(w):
                if self.walls[i][j]:
                    canvas[i, j] = wall_color
                elif (i, j) == self.start:
                    canvas[i, j] = start_color
                elif (i, j) == self.goal:
                    canvas[i, j] = goal_color
                elif (i, j) in solution_cells:
                    canvas[i, j] = solution_color
                elif show_explored and (i, j) in self.explored:
                    canvas[i, j] = explored_color

        if cell_size != 1:
            canvas = np.kron(canvas, np.ones((cell_size, cell_size, 1), dtype=np.uint8))   # Yeh har cell ko bada kar deta hai zoom ke liye

        return canvas

    def show(self, show_solution=True, show_explored=False, cell_size=20):
        arr = self._render_array(show_solution, show_explored, cell_size)
        plt.figure(figsize=(max(6, self.width*cell_size/50), max(4, self.height*cell_size/50)))   # Figure size adjust kar diya
        plt.imshow(arr, origin='upper', interpolation='nearest')   # Maze ko draw karna
        plt.axis('off')   # Axis hatana
        plt.show()   # Display karna


# ---------------- Run Solver ----------------
maze = Maze.from_file("maze.txt")   # Yeh maze.txt file se maze load kar raha hai
maze.solve(frontier="bfs")   # Yeh maze ko BFS algorithm se solve kar raha hai (DFS bhi try kar sakte ho)

print("Explored cells:", maze.num_explored)   # Yeh print karega total explored cells
print("Solution path:", maze.solution[0])   # Yeh print karega moves ka list (up, down, left, right)

maze.show(show_solution=True, show_explored=True)   # Yeh maze ko graphical form mein dikhayega

AttributeError: 'QueueFrontier' object has no attribute 'frontier'